# Uploading Annotations

In this tutorial, we explore some different options to upload annotations in Remo. We will:
- introduce the concept of Annotation set
- add annotations using a file format supported by Remo
- add annotations directly from code
- add annotations using custom files formats

We start off by creating a dataset and populating it with some images

In [1]:
%load_ext autoreload
%autoreload 2
import sys
# need to specify path to remo in notebook
local_path_to_repo =  '/home/andrea/Desktop/Projects/repo/remo-python'

sys.path.insert(0, local_path_to_repo)

import remo
import os
import pandas as pd

urls = ['https://remo-scripts.s3-eu-west-1.amazonaws.com/open_images_sample_dataset.zip']
my_dataset = remo.create_dataset(name = 'D1', urls = urls)


    (\(\ 
    (>':') Remo server is running: v0.3.10-77-g3ab8aa16
                


In [2]:
remo.list_datasets()

[Dataset 2 - 'hc_symbols',
 Dataset 3 - 'test',
 Dataset 4 - 'D1',
 Dataset 5 - 'test',
 Dataset 6 - 'D1']

In [3]:
my_dataset = remo.get_dataset(5)

In [2]:
images_dir = ['/home/andrea/Desktop/Projects/rem_work/open_images_sample_dataset']

my_dataset = remo.create_dataset(name='test', local_files = images_dir)

In [4]:
my_dataset.annotation_sets()

[]

In [5]:
my_annotation_file = ['/home/andrea/Desktop/Projects/rem_work/open_images_sample_dataset/open_sample.csv']
my_dataset.add_data(paths_to_upload=my_annotation_file, annotation_set_id=2)

Exception: Annotation set with ID = 2 not found. You can check the list of annotation sets in your dataset using dataset.annotation_sets()

## Annotation set

Annotation sets represent a collection of annotations for all the images within a Dataset.
An annotation set is characterized by a task (such as 'Object Detection') and a list of classes.

The advantage of grouping annotations in an Annotation Set is that it allows for high-level group operations on all the annotations, such:
- grouping classes together
- deleting objects of specific classes
- comparing of different annotations (such as ground truth vs prediction, or annotations coming from different annotators)


**Let's first create an empty annotation set with a predetermined list of classes**

In [28]:
my_classes = ['Airplane', 'Clothing', 'Dog', 'Fashion accessory', 'Food', 'Footwear', 'Fruit', 'Human arm', 
         'Human body', 'Human hand', 'Human leg', 'Mammal', 'Man', 'Person', 'Salad', 'Sports equipment', 'Trousers', 'Woman']

annotation_set = my_dataset.create_annotation_set(annotation_task = 'Object detection',
                                          name = 'Objects',
                                          classes = my_classes)

We can easily retrieve different annotation sets of a dataset

In [29]:
my_dataset.annotation_sets()

[Annotation set 17 - 'Objects', task: Object detection, #classes: 18]

## Add annotations from supported file format

If your annotations are already in a format supported by Remo, you can upload them right away using
my_dataset.add_data().

**As an example, let's see how to add some annotations from a CSV file**

In thiss case, annotations are stored in a CSV file in a format already supported by Remo. Class labels were encoded using [GoogleKnowledgeGraph](https://developers.google.com/knowledge-graph).

In [4]:
annotation_files=[os.getcwd() + '/assets/open_sample.csv']

df = pd.read_csv(annotation_files[0])
df.columns

Index(['ImageID', 'Source', 'LabelName', 'Confidence', 'XMin', 'XMax', 'YMin',
       'YMax', 'IsOccluded', 'IsTruncated', 'IsGroupOf', 'IsDepiction',
       'IsInside'],
      dtype='object')

When adding the file to the annotation set, remo automatically: 
- detects the class encoding and translates it into the corresponding labels
- only adds annotations for classes that are part of the existing annotation set

In [31]:
my_dataset.add_data(local_files=annotation_files, annotation_task = 'Object detection')

{'files_link_result': {'files uploaded': 0, 'annotations': 9, 'errors': []}}

In [32]:
my_dataset.get_annotation_statistics()

[{'AnnotationSet ID': 17,
  'AnnotationSet name': 'Objects',
  'n_images': 9,
  'n_classes': 18,
  'n_objects': 84,
  'top_3_classes': [{'name': 'Fruit', 'count': 27},
   {'name': 'Sports equipment', 'count': 12},
   {'name': 'Mammal', 'count': 7}],
  'creation_date': None,
  'last_modified_date': '2020-03-08T16:35:05.643905Z'}]

In [33]:
my_dataset.view()

Open http://localhost:8123/datasets/26


![dataset_added_annotation.jpeg](assets/dataset_added_annotation.jpeg) 

## Add annotations from custom formats

In case your annotations are in a custom format, as long as the task is one of those currently supported by Remo, it's still very easy to upload annotations from code.

**As an example, let's see how we can add annotations to a specific image from code**

This can be useful for instance to add model predictions as annotations or to tag specific images.

First, let's retrieve one image

In [37]:
images = my_dataset.images()
my_image = images[1]
print(my_image)
print('Resoultion: ', my_image.width, 'x', my_image.height)

Image: 936 - 000a1249af2bc5f0.jpg
Resoultion:  1024 x 678


**Now we can easily add annotations using add_annotations() method of the dataset class**

In [38]:
annotations = []

annotation = remo.Annotation()
annotation.img_filename = my_image.name
annotation.classes='Human hand'
annotation.bbox=[227, 284, 678, 674]
annotations.append(annotations)

annotation = remo.Annotation()
annotation.img_filename = my_image.name
annotation.classes='Human hand'
annotation.bbox=[311, 193, 607, 526]
annotations.append(annotations)

annotation = remo.Annotation()
annotation.img_filename = my_image.name
annotation.classes='Fashion accessory'
annotation.bbox=[496, 322, 544,370]
annotations.append(annotations)

ds.add_annotations(annotations)

In case your input data is in a custom file, you can write a parser to load annotations using the Annotation object.